# Recommender system using clustering
Collaborative filtering

In [1]:
# Importing necessary library
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
mouse = pd.read_csv('data/M-Mouse.csv', sep=";", error_bad_lines=False, encoding='latin-1')

c:\Users\sillydragon6\anaconda3\envs\books\lib\site-packages\IPython\core\interactiveshell.py:3457: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)
b'Skipping line 3: expected 6 fields, saw 11\n'


In [3]:
mouse.head()

,SKU,M-Title,Company,Image-URL-S,Image-URL-M,Image-URL-L
0,1,Rival 3,Steelseries,https://m.media-amazon.com/images/I/71EOtRi-c6...,https://m.media-amazon.com/images/I/71EOtRi-c6...,https://m.media-amazon.com/images/I/71EOtRi-c6...
1,4,M3,Keychron,https://m.media-amazon.com/images/I/61gB1njl6B...,https://m.media-amazon.com/images/I/61gB1njl6B...,https://m.media-amazon.com/images/I/61gB1njl6B...
2,5,Viper,Razer,https://m.media-amazon.com/images/I/71ECuJJK92...,https://m.media-amazon.com/images/I/71ECuJJK92...,https://m.media-amazon.com/images/I/71ECuJJK92...
3,6,Deathadder V3,Razer,https://m.media-amazon.com/images/I/51XO3LBZEJ...,https://m.media-amazon.com/images/I/51XO3LBZEJ...,https://m.media-amazon.com/images/I/51XO3LBZEJ...
4,7,G Pro Wireless,Logitech,https://m.media-amazon.com/images/I/51ySu55JzA...,https://m.media-amazon.com/images/I/51ySu55JzA...,https://m.media-amazon.com/images/I/51ySu55JzA...


In [4]:
len(mouse)
mouse.iloc[1]

SKU                                                            4
M-Title                                                       M3
Company                                                 Keychron
Image-URL-S    https://m.media-amazon.com/images/I/61gB1njl6B...
Image-URL-M    https://m.media-amazon.com/images/I/61gB1njl6B...
Image-URL-L    https://m.media-amazon.com/images/I/61gB1njl6B...
Name: 1, dtype: object

In [5]:
mouse.iloc[6]['Image-URL-L']

'https://m.media-amazon.com/images/I/61wKhxyyIbL._AC_UF894,1000_QL80_.jpg'

In [6]:
mouse.iloc[6]

SKU                                                            9
M-Title                                              X2 Wireless
Company                                                   Pulsar
Image-URL-S    https://m.media-amazon.com/images/I/61wKhxyyIb...
Image-URL-M    https://m.media-amazon.com/images/I/61wKhxyyIb...
Image-URL-L    https://m.media-amazon.com/images/I/61wKhxyyIb...
Name: 6, dtype: object

In [7]:
# !curl "http://images.amazon.com/images/P/0195153448.01.THUMBZZZ.jpg" --out.png
# !curl http://images.amazon.com/images/P/0060973129.01.THUMBZZZ.jpg --output some.jpg

In [8]:
mouse.shape

(8, 6)

In [9]:
mouse.columns

Index(['SKU', 'M-Title', 'Company', 'Image-URL-S', 'Image-URL-M',
       'Image-URL-L'],
      dtype='object')

#### Conclution:
Here Image URL columns is important for the poster. So, we will keep it

In [10]:
mouse = mouse[['SKU','M-Title', 'Company','Image-URL-L']]

In [11]:
mouse.head()

,SKU,M-Title,Company,Image-URL-L
0,1,Rival 3,Steelseries,https://m.media-amazon.com/images/I/71EOtRi-c6...
1,4,M3,Keychron,https://m.media-amazon.com/images/I/61gB1njl6B...
2,5,Viper,Razer,https://m.media-amazon.com/images/I/71ECuJJK92...
3,6,Deathadder V3,Razer,https://m.media-amazon.com/images/I/51XO3LBZEJ...
4,7,G Pro Wireless,Logitech,https://m.media-amazon.com/images/I/51ySu55JzA...


In [12]:
# Lets remane some wierd columns name
mouse.rename(columns={"M-Title":'title',
                     "Company":"company",
                     "Image-URL-L":"image_url"},inplace=True)

In [13]:
mouse.head()

,SKU,title,company,image_url
0,1,Rival 3,Steelseries,https://m.media-amazon.com/images/I/71EOtRi-c6...
1,4,M3,Keychron,https://m.media-amazon.com/images/I/61gB1njl6B...
2,5,Viper,Razer,https://m.media-amazon.com/images/I/71ECuJJK92...
3,6,Deathadder V3,Razer,https://m.media-amazon.com/images/I/51XO3LBZEJ...
4,7,G Pro Wireless,Logitech,https://m.media-amazon.com/images/I/51ySu55JzA...


In [14]:
# Now load the second dataframe

users = pd.read_csv('data/M-Users.csv', sep=";", error_bad_lines=False, encoding='latin-1')

c:\Users\sillydragon6\anaconda3\envs\books\lib\site-packages\IPython\core\interactiveshell.py:3457: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)


In [15]:
users.head()

,"ï»¿""User-ID""",Location,Age
0,1,new york,12
1,2,california,18
2,3,yukon territory,24
3,4,virginia,17
4,5,hants,30


In [16]:
users.shape

(10, 3)

In [17]:
# Lets remane some wierd columns name
users.rename(columns={"User-ID":'user_id',
                      'Location':'location',
                     "Age":'age'},inplace=True)

In [18]:
users.head(2)

,"ï»¿""User-ID""",location,age
0,1,new york,12
1,2,california,18


In [19]:
# Now load the third dataframe

ratings = pd.read_csv('data/M-Rating.csv', sep=";", error_bad_lines=False, encoding='latin-1')

c:\Users\sillydragon6\anaconda3\envs\books\lib\site-packages\IPython\core\interactiveshell.py:3457: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)


In [20]:
ratings.head()

,User-ID,SKU,Mouse-Rating
0,1,1,2
1,1,2,2
2,1,3,2
3,1,4,1
4,1,5,10


In [21]:
ratings.shape

(100, 3)

In [22]:
# Lets remane some wierd columns name
ratings.rename(columns={"User-ID":'user_id',
                      'Mouse-Rating':'rating'},inplace=True)

In [23]:
ratings.head(2)

,user_id,SKU,rating
0,1,1,2
1,1,2,2


### Conclution:
Now we have 3 dataframes
- mouse
- users
- ratings

In [24]:
print(mouse.shape, users.shape, ratings.shape, sep='\n')



(8, 4)
(10, 3)
(100, 3)


In [25]:
ratings['user_id'].value_counts()

1     10
2     10
3     10
4     10
5     10
6     10
7     10
8     10
9     10
10    10
Name: user_id, dtype: int64

In [26]:
ratings['user_id'].value_counts().shape

(10,)

In [27]:
ratings['user_id'].unique().shape

(10,)

In [28]:
# Lets store users who had at least rated more than 8 mouse
x = ratings['user_id'].value_counts() > 8

In [29]:
x[x].shape

(10,)

In [30]:
y= x[x].index

In [31]:
y

Int64Index([1, 2, 3, 4, 5, 6, 7, 8, 9, 10], dtype='int64')

In [32]:
ratings = ratings[ratings['user_id'].isin(y)]

In [33]:
ratings.head()

,user_id,SKU,rating
0,1,1,2
1,1,2,2
2,1,3,2
3,1,4,1
4,1,5,10


In [34]:
ratings.shape

(100, 3)

In [35]:
# Now join ratings with mouse

ratings_with_mouse = ratings.merge(mouse, on='SKU')

In [36]:
ratings_with_mouse.head()

,user_id,SKU,rating,title,company,image_url
0,1,1,2,Rival 3,Steelseries,https://m.media-amazon.com/images/I/71EOtRi-c6...
1,2,1,9,Rival 3,Steelseries,https://m.media-amazon.com/images/I/71EOtRi-c6...
2,3,1,3,Rival 3,Steelseries,https://m.media-amazon.com/images/I/71EOtRi-c6...
3,4,1,1,Rival 3,Steelseries,https://m.media-amazon.com/images/I/71EOtRi-c6...
4,5,1,3,Rival 3,Steelseries,https://m.media-amazon.com/images/I/71EOtRi-c6...


In [37]:
ratings_with_mouse.shape

(80, 6)

In [38]:
number_rating = ratings_with_mouse.groupby('title')['rating'].count().reset_index()

In [39]:
number_rating.head()

,title,rating
0,Deathadder V3,10
1,G Pro Wireless,10
2,G502,10
3,M3,10
4,Model O,10


In [40]:
number_rating.rename(columns={'rating':'num_of_rating'},inplace=True)

In [41]:
number_rating.head()

,title,num_of_rating
0,Deathadder V3,10
1,G Pro Wireless,10
2,G502,10
3,M3,10
4,Model O,10


In [42]:
final_rating = ratings_with_mouse.merge(number_rating, on='title')

In [43]:
final_rating.head()

,user_id,SKU,rating,title,company,image_url,num_of_rating
0,1,1,2,Rival 3,Steelseries,https://m.media-amazon.com/images/I/71EOtRi-c6...,10
1,2,1,9,Rival 3,Steelseries,https://m.media-amazon.com/images/I/71EOtRi-c6...,10
2,3,1,3,Rival 3,Steelseries,https://m.media-amazon.com/images/I/71EOtRi-c6...,10
3,4,1,1,Rival 3,Steelseries,https://m.media-amazon.com/images/I/71EOtRi-c6...,10
4,5,1,3,Rival 3,Steelseries,https://m.media-amazon.com/images/I/71EOtRi-c6...,10


In [44]:
final_rating.shape

(80, 7)

In [45]:
# Lets take those mouse which got at least 5 rating of user

final_rating = final_rating[final_rating['num_of_rating'] >= 5]

In [46]:
final_rating.head()

,user_id,SKU,rating,title,company,image_url,num_of_rating
0,1,1,2,Rival 3,Steelseries,https://m.media-amazon.com/images/I/71EOtRi-c6...,10
1,2,1,9,Rival 3,Steelseries,https://m.media-amazon.com/images/I/71EOtRi-c6...,10
2,3,1,3,Rival 3,Steelseries,https://m.media-amazon.com/images/I/71EOtRi-c6...,10
3,4,1,1,Rival 3,Steelseries,https://m.media-amazon.com/images/I/71EOtRi-c6...,10
4,5,1,3,Rival 3,Steelseries,https://m.media-amazon.com/images/I/71EOtRi-c6...,10


In [47]:
final_rating.shape

(80, 7)

In [48]:
# lets drop the duplicates
final_rating.drop_duplicates(['user_id','title'],inplace=True)

In [49]:
final_rating.shape

(80, 7)

In [50]:
# Lets create a pivot table
mouse_pivot = final_rating.pivot_table(columns='user_id', index='title', values= 'rating')

In [51]:
mouse_pivot

user_id,1,2,3,4,5,6,7,8,9,10
title,,,,,,,,,,
Deathadder V3,2,3,9,5,7,6,6,10,8,5
G Pro Wireless,10,1,10,6,6,5,10,6,6,6
G502,4,8,4,1,10,2,6,1,1,1
M3,1,4,7,3,9,6,4,10,6,2
Model O,5,7,2,10,3,6,8,6,2,5
Rival 3,2,9,3,1,3,1,4,8,5,5
Viper,10,4,4,1,7,6,6,3,4,8
X2 Wireless,10,6,5,5,2,10,7,10,7,8


In [52]:
mouse_pivot.shape

(8, 10)

In [53]:
mouse_pivot.fillna(0, inplace=True)

In [54]:
mouse_pivot

user_id,1,2,3,4,5,6,7,8,9,10
title,,,,,,,,,,
Deathadder V3,2,3,9,5,7,6,6,10,8,5
G Pro Wireless,10,1,10,6,6,5,10,6,6,6
G502,4,8,4,1,10,2,6,1,1,1
M3,1,4,7,3,9,6,4,10,6,2
Model O,5,7,2,10,3,6,8,6,2,5
Rival 3,2,9,3,1,3,1,4,8,5,5
Viper,10,4,4,1,7,6,6,3,4,8
X2 Wireless,10,6,5,5,2,10,7,10,7,8


# Training Model

In [55]:
from scipy.sparse import csr_matrix

In [56]:
mouse_sparse = csr_matrix(mouse_pivot)

In [57]:
type(mouse_sparse)

scipy.sparse.csr.csr_matrix

In [58]:
# Now import our clustering algoritm which is Nearest Neighbors this is an unsupervised ml algo
from sklearn.neighbors import NearestNeighbors
model = NearestNeighbors(algorithm= 'brute')

In [59]:
model.fit(mouse_sparse)

NearestNeighbors(algorithm='brute')

In [60]:
distance, suggestion = model.kneighbors(mouse_pivot.iloc[5,:].values.reshape(1,-1), n_neighbors=6 )

In [61]:
distance

array([[ 0.        , 11.        , 11.87434209, 12.08304597, 12.20655562,
        13.03840481]])

In [62]:
suggestion

array([[5, 3, 2, 0, 4, 6]], dtype=int64)

In [63]:
mouse_pivot.iloc[5,:]

user_id
1     2
2     9
3     3
4     1
5     3
6     1
7     4
8     8
9     5
10    5
Name: Rival 3, dtype: int64

In [64]:
for i in range(len(suggestion)):
    print(mouse_pivot.index[suggestion[i]])

Index(['Rival 3', 'M3', 'G502', 'Deathadder V3', 'Model O', 'Viper'], dtype='object', name='title')


In [65]:
mouse_pivot.index[3]

'M3'

In [66]:
#keeping mouse name
mouse_names = mouse_pivot.index

In [67]:
mouse_names[2]

'G502'

In [68]:
np.where(mouse_pivot.index == 'Viper')[0][0]

6

# find url

In [69]:
# final_rating['title'].value_counts()
ids = np.where(final_rating['title'] == "Rival 3")[0][0]

In [70]:
final_rating.iloc[ids]['image_url']

'https://m.media-amazon.com/images/I/71EOtRi-c6L._AC_UF894,1000_QL80_.jpg'

In [71]:
mouse_name = []
for mouse_id in suggestion:
    mouse_name.append(mouse_pivot.index[mouse_id])
    
    

In [72]:
mouse_name[0]

Index(['Rival 3', 'M3', 'G502', 'Deathadder V3', 'Model O', 'Viper'], dtype='object', name='title')

In [73]:
ids_index = []
for name in mouse_name[0]: 
    ids = np.where(final_rating['title'] == name)[0][0]
    ids_index.append(ids)

In [74]:
for idx in ids_index:
    url = final_rating.iloc[idx]['image_url']
    print(url)

https://m.media-amazon.com/images/I/71EOtRi-c6L._AC_UF894,1000_QL80_.jpg
https://m.media-amazon.com/images/I/61gB1njl6BL._AC_UF1000,1000_QL80_.jpg
https://m.media-amazon.com/images/I/71WHWIq2VBL._AC_UF894,1000_QL80_.jpg
https://m.media-amazon.com/images/I/51XO3LBZEJL._AC_UF894,1000_QL80_.jpg
https://m.media-amazon.com/images/I/71SmrOjHDFL.jpg
https://m.media-amazon.com/images/I/71ECuJJK92L._AC_UF894,1000_QL80_.jpg


In [75]:
import pickle
pickle.dump(model,open('artifacts/mrmodel.pkl','wb'))
pickle.dump(mouse_names,open('artifacts/mouse_names.pkl','wb'))
pickle.dump(final_rating,open('artifacts/m_final_rating.pkl','wb'))
pickle.dump(mouse_pivot,open('artifacts/mouse_pivot.pkl','wb'))

# Testing model

In [76]:
def recommend_book(mouse_name):
    mouse_id = np.where(mouse_pivot.index == mouse_name)[0][0]
    distance, suggestion = model.kneighbors(mouse_pivot.iloc[mouse_id,:].values.reshape(1,-1), n_neighbors=6 )
    
    for i in range(len(suggestion)):
            mouse = mouse_pivot.index[suggestion[i]]
            for j in mouse:
                if j == mouse_name:
                    print(f"You searched '{mouse_name}'\n")
                    print("The suggestion are: \n")
                else:
                    print(j)

In [77]:
mouse_name = "M3"
recommend_book(mouse_name)

You searched 'M3'

The suggestion are: 

Deathadder V3
Rival 3
G502
G Pro Wireless
Viper
